### Things to get from data
1. time tkane for introduction trial ([4,2,3,5,1])
2. time taken for each imitation trial
3. initial states, sequence of comparison, final state for each test trial
4. choices for demo trials
5. initial states, sequence of comparison, final state for each demo trial
6. description

In [1]:
import pandas as pd
import numpy as np
import copy
import glob 
import os 

In [2]:
#calculates sequence of comparisons, whether they swap and number of least necessary comparisons for an array with a certain order (for example: 6 4 3 1 5 2)
#_f means forward and _b means backward
def imit_seq(array):
    aa=copy.copy(array)
    comp_seq=[]
    swap_or_not=[]
    ii_f=0
    nn=0
    while nn<30:
        
        if aa[ii_f]<aa[ii_f+1]: #correct ofer and keep going forward
            comp_seq.append([ii_f,ii_f+1])
            swap_or_not.append(0)
            nn+=1
            ii_f+=1
        else: #switch when going forward
            comp_seq.append([ii_f,ii_f+1])
            aa[ii_f],aa[ii_f+1]=aa[ii_f+1],aa[ii_f]
            swap_or_not.append(1)
            nn+=1

            ii_b=ii_f
            ii_f+=1

            while nn<30:
                if ii_b==0: #back to position zero
                    break
                if aa[ii_b-1]>aa[ii_b]: #switch and keep going back
                    comp_seq.append([ii_b-1,ii_b])
                    swap_or_not.append(1)
                    aa[ii_b-1],aa[ii_b]=aa[ii_b],aa[ii_b-1]
                    nn+=1
                    ii_b-=1
                if aa[ii_b-1]<aa[ii_b]: #stop going back
                    if ii_b==0:
                        break
                    else:
                        comp_seq.append([ii_b-1,ii_b])
                        swap_or_not.append(0)
                        nn+=1
                        break
        if (aa==np.array(range(1, len(array)+1, 1))).all():
            if np.array([len(array)-2,len(array)-1]) in np.array(comp_seq):
                break
    return array,nn,comp_seq,swap_or_not




# #verify n_nece_compare
# arr_test=np.array([6,4,3,1,5,2]) #13
# arr_test=np.array([1, 2, 3, 4, 8, 7, 6, 5])
# # arr_test=np.array([2,1,4,3,6,5]) #7
# # arr_test=np.array([4,6,3,2,5,1]) #13

# print(imit_seq(arr_test))

In [15]:
# Find Cognition.run datafiles corresponding to prolific ID
pilot_3_pid=["6793751badf02fd55a1f0fb9","66a0172986186ec228714d51","66986c9ad97d2e5ff07329c1","65610d3b7a06953204bbbb62","677527596d51edae18442b1e","6595ddebdbc6d36b1ef5462a","6774ed8fea9a49a640664efb","576704bffdff7a0001c12890","679359525d6dac68ab9d189f","6629517257384969d8995d78","5a8cba1a8fe2dc00010643f0"]
path_pilot_3='C:/Users/guoqi/GITHUB_QGUO/SSL_teaching/Pilot Data/Data/Pilot/Pilot_3_Jan25_10_Participants/'
csv_files = glob.glob(os.path.join(path_pilot_3, '*.csv')) 
# print(csv_files)
run_id_pilot_3=[]
p_id_pilot_3=[]
for csv in csv_files:
    data = pd.read_csv(csv) 
    # print(data['run_id'][0])
    if 'PROLIFIC_PID'not in data:
        # print("none")
        continue
    elif 'phase' not in data:
        # print("none")
        continue
    else:
        data_useful=data[[ 'run_id','PROLIFIC_PID']]
        pid=data_useful['PROLIFIC_PID'][0]
        pid_4=pid[-4:] #last four digit of prolific id
    # print(data_useful['PROLIFIC_PID'])
        if pid in pilot_3_pid:
            run_id_pilot_3.append(data_useful['run_id'][0])
            p_id_pilot_3.append(pid_4)
print(run_id_pilot_3)
print(p_id_pilot_3)

[30, 31, 32, 35, 36, 37, 42, 43, 44, 51, 52]
['0fb9', '4d51', '29c1', 'bb62', '2b1e', '462a', '4efb', '2890', '189f', '5d78', '43f0']


In [76]:
# data=pd.read_csv(
#     'data/pilot/Pilot_1_Jan20_3_Participants/14.csv'
# )
# data=pd.read_csv(
#     'data/pilot/Pilot_2_Jan22_5_Participants/14.csv'
# )
# data=pd.read_csv(
#     path_pilot_3+str(run_id_pilot_3[7])+'.csv'
# )
# data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
# data_intro_trial=data_useful[['time_elapsed','stimulus','response'][]]

# Test_label = [&quot;Test 0&quot;, &quot;y&quot;, &quot;z&quot;]
for pp in run_id_pilot_3:
    data=pd.read_csv(path_pilot_3+str(pp)+'.csv')
    data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
        
    p_gnome_all=[]
    for ii in range(0,10):
        indices_test=data_useful[data_useful['phase'] == "Test "+str(ii)].index.to_numpy()
        fin_order_test=data_useful['phase'][max(indices_test)+1]
        fin_order_test=fin_order_test[13:].split(",")
        fin_order_test=[int(x) for x in fin_order_test]==[1,2,3,4,5] or [int(x) for x in fin_order_test]==[1,2,3,4,5,6,7,8]
    
        ini_order_test=data_useful['phase'][min(indices_test)-1]
        ini_order_test=ini_order_test[15:].split(",")
        ini_order_test=[int(x)+1 for x in ini_order_test]
    
        comp_test = [data['response'][i] for i in indices_test]
        comp_test=[int(x) for x in comp_test]
        comp_test=comp_test[:-1]
        if len(comp_test)%2==1:
            comp_test=comp_test[:-1]
        n_comp_ptcp=int(len(comp_test)/2) #number of comparison made by participants
        comp_test=np.reshape(comp_test, (n_comp_ptcp, 2)).tolist()
        # print(comp_test)
        for row in comp_test:
            if row[0]==row[1]:
                comp_test.remove(row)
        for row in comp_test:
            if row[0]>row[1]:
                cache=row[0]
                row[0]=row[1]
                row[1]=cache
        # print(fin_order_test)
        # print(comp_test)
    
        
        windows_test={}
        for jj in range(len(comp_test),1,-1):
            for kk in range(0,len(comp_test)-jj+1):
                if jj not in windows_test.keys():
                    windows_test[jj]=[comp_test[kk:(kk+jj)]]
                else:
                    windows_test[jj].append(comp_test[kk:(kk+jj)])
        # print(windows_test)
        
    
        array_cache,nn_cache,comp_seq_true,swap_or_not_cache=imit_seq(np.array(ini_order_test))
        # print(comp_seq_true)
        
        windows_true={}
        for jj in range(len(comp_seq_true),1,-1):
            for kk in range(0,len(comp_seq_true)-jj+1):
                if jj not in windows_true.keys():
                    windows_true[jj]=[comp_seq_true[kk:(kk+jj)]]
                else:
                    windows_true[jj].append(comp_seq_true[kk:(kk+jj)])
        # print(windows_true)
    
        common_size=min(len(comp_seq_true),len(comp_test))
        n_comp_true=len(comp_seq_true) #number of comparison for correct gnome
        sim={}
        for mm in range(2,n_comp_true+1):
            sim[mm]=0
            if mm<=common_size:
                win_test=windows_test[mm]
                win_true=windows_true[mm]
                for comp_test in win_test:
                    if comp_test in win_true:
                        sim[mm]+=1
            else:
                sim[mm]=0
        # print(sim)
    
        sum_test=0
        sum_true=0
        for key in sim.keys():
            # sum_true+=key*(common_size-key+1)*(1/2)**(common_size-key)
            sum_true+=key
            if sim[key]>0:
                # sum_test=key
                # sum_test+=key*sim[key]*(1/2)**(common_size-key)
                sum_test+=key*sim[key]/len(windows_true[key])
        if sum_test>sum_true:
            sum_test=sum_true
        # print(sum_test/sum_true)       
    
    
        # for ii in range(len(comp_seq_true),1,-1):
        #     sum_true+=ii*(common_size-ii+1)*(1/2)**(common_size-ii)
    
        p_gnome=sum_test/sum_true
        p_gnome_all.append(p_gnome)
    # p_gnome_all=np.array(p_gnome_all)
    # print(p_gnome_all)
    p_half_criteria=np.quantile(p_gnome_all, 0.5)
    
    p_half=[p for p in p_gnome_all if p>=p_half_criteria]
    p_half_mean=np.average(p_half)
    
    print(p_half_mean)
    # p_gnome_all[p_gnome_all>=p_half]
        # print(sum_true,sum_test)

            
        



# index_crclm=data_useful[data_useful['trial_type'] == "survey-multi-choice"].index.to_numpy()

# crclm=data_useful['response'][index_crclm]
# index_ling=data_useful[data_useful['trial_type'] == "survey-text"].index.to_numpy()
# ling=data_useful['response'][index_ling]
# # crclm=crclm.split(",")
# print(crclm)
# print(ling)

0.5779529680845471
0.187966530804154
0.29730031745749474
1.0
0.12070646637313302
1.0
0.1297251158600365
0.22247481277841707
1.0
0.0356158616697453
0.7116608946608947


In [103]:
data=pd.read_csv(
    'data/pilot/Pilot_1_Jan20_3_Participants/14.csv'
)
data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
# data_intro_trial=data_useful[['time_elapsed','stimulus','response'][]]

# Test_label = [&quot;Test 0&quot;, &quot;y&quot;, &quot;z&quot;]
for ii in range(0,10):
    indices_test=data_useful[data_useful['phase'] == "Test "+str(ii)].index.to_numpy()
    fin_order_test=data_useful['phase'][max(indices_test)+1]
    fin_order_test=fin_order_test[13:].split(",")
    fin_order_test=[int(x) for x in fin_order_test]==[1,2,3,4,5] or [int(x) for x in fin_order_test]==[1,2,3,4,5,6,7,8]

    ini_order_test=data_useful['phase'][min(indices_test)-1]
    ini_order_test=ini_order_test[15:].split(",")
    ini_order_test=[int(x)+1 for x in ini_order_test]

    comp_test_1 = [data['response'][i] for i in indices_test]
    comp_test_1=[int(x) for x in comp_test_1]
    comp_test_1=comp_test_1[:-1]
    if len(comp_test_1)%2==1:
        comp_test_1=comp_test_1[:-1]
    n_comp_ptcp=int(len(comp_test_1)/2) #number of comparison made by participants
    comp_test_1=np.reshape(comp_test_1, (n_comp_ptcp, 2)).tolist()
    for row in comp_test_1:
        if row[0]==row[1]:
            comp_test_1.remove(row)
    print(fin_order_test)
    print(comp_test_1)

    array_cache,nn_cache,comp_seq_true,swap_or_not_cache=imit_seq(np.array(ini_order_test))
    print(comp_seq_true)

index_crclm=data_useful[data_useful['trial_type'] == "survey-multi-choice"].index.to_numpy()

crclm=data_useful['response'][index_crclm]
index_ling=data_useful[data_useful['trial_type'] == "survey-text"].index.to_numpy()
ling=data_useful['response'][index_ling]
# crclm=crclm.split(",")
print(crclm)
print(ling)

True
[[0, 1], [1, 2], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
[[0, 1], [1, 2], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
True
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [2, 3], [5, 6], [4, 5], [3, 4], [2, 3], [6, 7], [5, 6]]
[[0, 1], [1, 2], [0, 1], [2, 3], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [2, 3], [5, 6], [4, 5], [3, 4], [2, 3], [6, 7], [5, 6]]
True
[[0, 1], [1, 2], [0, 1], [1, 2], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
True
[[0, 1], [1, 2], [0, 1], [2, 3], [2, 1], [1, 0], [3, 4], [4, 5], [4, 3], [5, 6], [5, 4], [4, 3], [3, 2], [2, 1], [1, 0], [7, 6], [6, 5], [5, 4]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [4, 5], [3, 4], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1], [6, 7], [5, 6], [4, 5]]
True
[[0, 1], [1, 0], [1, 2], [2, 3], [2, 1], [3, 4], [3, 2]]
[[0, 1], [1, 2], [2, 3], [1, 2

In [104]:
data=pd.read_csv(
    'data/pilot/Pilot_1_Jan20_3_Participants/13.csv'
)
data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
# data_intro_trial=data_useful[['time_elapsed','stimulus','response'][]]

# Test_label = [&quot;Test 0&quot;, &quot;y&quot;, &quot;z&quot;]
for ii in range(0,10):
    indices_test=data_useful[data_useful['phase'] == "Test "+str(ii)].index.to_numpy()
    fin_order_test=data_useful['phase'][max(indices_test)+1]
    fin_order_test=fin_order_test[13:].split(",")
    fin_order_test=[int(x) for x in fin_order_test]==[1,2,3,4,5] or [int(x) for x in fin_order_test]==[1,2,3,4,5,6,7,8]

    ini_order_test=data_useful['phase'][min(indices_test)-1]
    ini_order_test=ini_order_test[15:].split(",")
    ini_order_test=[int(x)+1 for x in ini_order_test]

    comp_test_1 = [data['response'][i] for i in indices_test]
    comp_test_1=[int(x) for x in comp_test_1]
    comp_test_1=comp_test_1[:-1]
    if len(comp_test_1)%2==1:
        comp_test_1=comp_test_1[:-1]
    n_comp_ptcp=int(len(comp_test_1)/2) #number of comparison made by participants
    comp_test_1=np.reshape(comp_test_1, (n_comp_ptcp, 2)).tolist()
    for row in comp_test_1:
        if row[0]==row[1]:
            comp_test_1.remove(row)
    print(fin_order_test)
    print(comp_test_1)

    array_cache,nn_cache,comp_seq_true,swap_or_not_cache=imit_seq(np.array(ini_order_test))
    print(comp_seq_true)

index_crclm=data_useful[data_useful['trial_type'] == "survey-multi-choice"].index.to_numpy()

crclm=data_useful['response'][index_crclm]
index_ling=data_useful[data_useful['trial_type'] == "survey-text"].index.to_numpy()
ling=data_useful['response'][index_ling]
# crclm=crclm.split(",")
print(crclm)
print(ling)

False
[[2, 3], [0, 1]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [3, 4]]
False
[[0, 1], [1, 2], [1, 0], [0, 3], [3, 4], [4, 5], [7, 6], [1, 0]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [2, 3], [5, 6], [4, 5], [3, 4], [6, 7], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1]]
True
[[0, 1], [1, 2], [1, 0], [2, 3], [2, 1], [3, 4], [4, 3], [0, 1]]
[[0, 1], [1, 2], [0, 1], [2, 3], [3, 4]]
False
[[0, 1], [1, 2], [3, 4], [3, 2], [1, 0], [6, 5], [5, 4], [6, 7], [7, 6], [3, 2], [1, 0]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [3, 4], [2, 3], [4, 5], [5, 6], [4, 5], [3, 4], [6, 7], [5, 6], [4, 5], [3, 4]]
True
[[0, 1], [1, 2], [2, 3], [3, 4], [1, 0]]
[[0, 1], [1, 2], [0, 1], [2, 3], [3, 4], [2, 3]]
False
[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [1, 0], [1, 3], [5, 5], [6, 7]]
[[0, 1], [1, 2], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [2, 3], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1], [6, 7], [5, 6], [4

In [122]:
# second pilot file names :20_cd36  21_6dcf  24_521c    25_dc63
data=pd.read_csv(
    'data/pilot/Pilot_2_Jan22_5_Participants/21_6dcf.csv'
)
data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
# data_intro_trial=data_useful[['time_elapsed','stimulus','response'][]]

# Test_label = [&quot;Test 0&quot;, &quot;y&quot;, &quot;z&quot;]
for ii in range(0,10):
    indices_test=data_useful[data_useful['phase'] == "Test "+str(ii)].index.to_numpy()
    fin_order_test=data_useful['phase'][max(indices_test)+1]
    fin_order_test=fin_order_test[13:].split(",")
    fin_order_test=[int(x) for x in fin_order_test]==[1,2,3,4,5] or [int(x) for x in fin_order_test]==[1,2,3,4,5,6,7,8]

    ini_order_test=data_useful['phase'][min(indices_test)-1]
    ini_order_test=ini_order_test[15:].split(",")
    ini_order_test=[int(x)+1 for x in ini_order_test]

    comp_test_1 = [data['response'][i] for i in indices_test]
    comp_test_1=[int(x) for x in comp_test_1]
    comp_test_1=comp_test_1[:-1]
    if len(comp_test_1)%2==1:
        comp_test_1=comp_test_1[:-1]
    n_comp_ptcp=int(len(comp_test_1)/2) #number of comparison made by participants
    comp_test_1=np.reshape(comp_test_1, (n_comp_ptcp, 2)).tolist()
    for row in comp_test_1:
        if row[0]==row[1]:
            comp_test_1.remove(row)
    print(fin_order_test)
    print(comp_test_1)

    array_cache,nn_cache,comp_seq_true,swap_or_not_cache=imit_seq(np.array(ini_order_test))
    print(comp_seq_true)

index_crclm=data_useful[data_useful['trial_type'] == "survey-multi-choice"].index.to_numpy()

crclm=data_useful['response'][index_crclm]
index_ling=data_useful[data_useful['trial_type'] == "survey-text"].index.to_numpy()
ling=data_useful['response'][index_ling]
crclm=np.array(crclm)
# crclm=


print(crclm)
ling=np.array(ling)
ling=ling[0][7:]
ling=ling[:-2]

print(ling)

False
[[0, 1]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
True
[[0, 1], [0, 1], [1, 2], [0, 1], [0, 1], [2, 3], [2, 1], [1, 0], [3, 4], [3, 2], [2, 1], [1, 0], [4, 5], [4, 3], [5, 6], [5, 4], [4, 3], [3, 2], [2, 1], [1, 0], [6, 7], [6, 5]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1], [6, 7], [5, 6]]
False
[[0, 1], [1, 2], [2, 3], [3, 2], [2, 1], [1, 0], [3, 4], [3, 2], [2, 1], [1, 0]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
False
[[0, 1], [1, 2], [1, 0], [2, 3], [3, 4], [3, 2], [4, 5], [4, 3], [3, 5], [6, 5], [5, 4], [4, 3], [3, 2], [2, 1], [1, 0], [0, 1], [5, 4], [5, 6], [6, 5], [6, 7], [6, 5], [5, 4]]
[[0, 1], [1, 2], [0, 1], [2, 3], [3, 4], [2, 3], [4, 5], [3, 4], [2, 3], [1, 2], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1], [6, 7], [5, 6], [4, 5]]
True
[[0, 1], [1, 2], [2, 3], [2, 1], [1, 0], [2, 3], [3,

In [5]:
# third pilot file names :20_cd36  21_6dcf  24_521c    25_dc63
data=pd.read_csv(
    'Data/Pilot/Pilot_3_Jan 25_10_Participants/31.csv'
)
data_useful=data[['trial_type','time_elapsed', 'run_id','PROLIFIC_PID','STUDY_ID','stimulus','response','phase','question_order']]
# data_intro_trial=data_useful[['time_elapsed','stimulus','response'][]]

# Test_label = [&quot;Test 0&quot;, &quot;y&quot;, &quot;z&quot;]
for ii in range(0,10):
    indices_test=data_useful[data_useful['phase'] == "Test "+str(ii)].index.to_numpy()
    fin_order_test=data_useful['phase'][max(indices_test)+1]
    fin_order_test=fin_order_test[13:].split(",")
    fin_order_test=[int(x) for x in fin_order_test]==[1,2,3,4,5] or [int(x) for x in fin_order_test]==[1,2,3,4,5,6,7,8]

    ini_order_test=data_useful['phase'][min(indices_test)-1]
    ini_order_test=ini_order_test[15:].split(",")
    ini_order_test=[int(x)+1 for x in ini_order_test]

    comp_test_1 = [data['response'][i] for i in indices_test]
    comp_test_1=[int(x) for x in comp_test_1]
    comp_test_1=comp_test_1[:-1]
    if len(comp_test_1)%2==1:
        comp_test_1=comp_test_1[:-1]
    n_comp_ptcp=int(len(comp_test_1)/2) #number of comparison made by participants
    comp_test_1=np.reshape(comp_test_1, (n_comp_ptcp, 2)).tolist()
    for row in comp_test_1:
        if row[0]==row[1]:
            comp_test_1.remove(row)
    print(fin_order_test)
    print(comp_test_1)

    array_cache,nn_cache,comp_seq_true,swap_or_not_cache=imit_seq(np.array(ini_order_test))
    print(comp_seq_true)

index_crclm=data_useful[data_useful['trial_type'] == "survey-multi-choice"].index.to_numpy()

crclm=data_useful['response'][index_crclm]
index_ling=data_useful[data_useful['trial_type'] == "survey-text"].index.to_numpy()
ling=data_useful['response'][index_ling]
crclm=np.array(crclm)
# crclm=


print(crclm)
ling=np.array(ling)
ling=ling[0][7:]
ling=ling[:-2]

print(ling)

True
[[0, 1], [1, 2], [2, 0], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2]]
False
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [4, 5], [3, 4], [2, 3], [5, 6], [6, 7], [5, 6], [5, 4]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [2, 3], [5, 6], [6, 7], [5, 6], [4, 5]]
False
[[0, 1], [1, 2], [2, 3], [3, 4], [2, 3], [1, 2]]
[[0, 1], [1, 2], [0, 1], [2, 3], [1, 2], [0, 1], [3, 4], [2, 3], [1, 2]]
True
[[0, 1], [1, 2], [2, 3], [1, 2], [3, 4], [2, 3], [1, 2], [0, 1], [1, 2], [2, 3], [4, 5], [3, 4], [5, 6], [5, 4], [3, 4], [2, 3], [1, 2], [0, 1], [0, 1], [1, 2], [3, 4], [5, 6], [6, 7], [5, 6]]
[[0, 1], [1, 2], [2, 3], [1, 2], [3, 4], [2, 3], [1, 2], [0, 1], [4, 5], [3, 4], [5, 6], [4, 5], [3, 4], [2, 3], [1, 2], [0, 1], [6, 7], [5, 6]]
True
[[0, 1], [1, 2], [2, 3], [0, 1], [2, 3], [3, 4], [2, 3], [1, 2], [0, 1]]
[[0, 1], [1,